In [1]:
import os
from time import sleep
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import warnings
import random

warnings.filterwarnings(action='ignore')

In [2]:
gu_list = ['중구 중앙동', '중구 동광동', '중구 대청동', '중구 보수동', '중구 부평동',
          '중구 광복동', '중구 남포동', '중구 영주동', '서구 동대신동', '서구 서대신동',
          '서구 부민동', '서구 아미동', '서구 초장동', '서구 충무동', '서구 남부민동',
          '서구 암남동', '동구 초량동', '동구 수정동', '동구 좌천동', '동구 범일동',
          '영도구 남항동', '영도구 영선동', '영도구 신선동', '영도구 봉래동', '영도구 청학동',
          '영도구 동삼동', '부산진구 부전동', '부산진구 연지동', '부산진구 초읍동',
          '부산진구 양정동', '부산진구 전포동', '부산진구 부암동', '부산진구 당감동',
          '부산진구 가야동', '부산진구 개금동', '부산진구 범천동', '동래구 수민동',
          '동래구 복산동', '동래구 명륜동', '동래구 온천동', '동래구 사직동', '동래구 안락동',
          '동래구 명장동', '남구 대연동', '남구 용호동', '남구 용당동', '남구 감만동', '남구 우암동',
          '남구 문현동', '북구 구포동', '북구 금곡동', '북구 화명동', '북구 덕천동', '북구 만덕동',
          '해운대구 우동', '해운대구 중동', '해운대구 좌동', '해운대구 송정동', '해운대구 반여동',
          '해운대구 반송동', '해운대구 재송동', '사하구 괴정동', '사하구 당리동', '사하구 감천동',
          '사하구 다대동', '사하구 구평동', '사하구 장림동', '사하구 신평동', '사하구 하단동',
          '금정구 서동', '금정구 금사동', '금정구 부곡동', '금정구 장전동', '금정구 남산동', '금정구 구서동',
          '금정구 청룡동', '금정구 선동', '금정구 금성동', '강서구 대저동', '강서구 강동동',
          '강서구 명지동', '강서구 가락동', '강서구 녹산동', '강서구 천성동', '연제구 거제동',
          '연제구  연산동', '수영구 남천동', '수영구 수영동', '수영구 망미동', '수영구 광안동', '수영구 민락동',
          '사상구 삼락동', '사상구 모라동', '사상구 덕포동', '사상구 쾌법동', '사상구 감전동', '사상구 주례동', '사상구 학장동', '사상구 엄궁동',
          '기장군 기장읍', '기장군 장안읍', '기장군 정관읍', '기장군 일광면', '기장군 철마면']

- 중구 보수동 해야함
- 부평동 분식까지함
- 광복동 분식까지함
- 중구 남포동 한식

In [3]:
categorys = ['간식', '분식', '뷔페', '술집', '아시아음식', '양식', '일식', '중식', '패스트푸드', '패밀리레스토랑', '피자', '치킨', '한식', '카페']

In [10]:
gu_list = ['중구 남포동', '중구 영주동', '서구 동대신동', '서구 서대신동',
          '서구 부민동', '서구 아미동', '서구 초장동', '서구 충무동', '서구 남부민동',
          '서구 암남동', '동구 초량동', '동구 수정동', '동구 좌천동', '동구 범일동',
          '영도구 남항동', '영도구 영선동', '영도구 신선동', '영도구 봉래동', '영도구 청학동',
          '영도구 동삼동', '부산진구 부전동', '부산진구 연지동', '부산진구 초읍동',
          '부산진구 양정동', '부산진구 전포동', '부산진구 부암동', '부산진구 당감동',
          '부산진구 가야동', '부산진구 개금동', '부산진구 범천동', '동래구 수민동',
          '동래구 복산동', '동래구 명륜동', '동래구 온천동', '동래구 사직동', '동래구 안락동',
          '동래구 명장동', '남구 대연동', '남구 용호동', '남구 용당동', '남구 감만동', '남구 우암동',
          '남구 문현동', '북구 구포동', '북구 금곡동', '북구 화명동', '북구 덕천동', '북구 만덕동',
          '해운대구 우동', '해운대구 중동', '해운대구 좌동', '해운대구 송정동', '해운대구 반여동',
          '해운대구 반송동', '해운대구 재송동', '사하구 괴정동', '사하구 당리동', '사하구 감천동',
          '사하구 다대동', '사하구 구평동', '사하구 장림동', '사하구 신평동', '사하구 하단동',
          '금정구 서동', '금정구 금사동', '금정구 부곡동', '금정구 장전동', '금정구 남산동', '금정구 구서동',
          '금정구 청룡동', '금정구 선동', '금정구 금성동', '강서구 대저동', '강서구 강동동',
          '강서구 명지동', '강서구 가락동', '강서구 녹산동', '강서구 천성동', '연제구 거제동',
          '연제구  연산동', '수영구 남천동', '수영구 수영동', '수영구 망미동', '수영구 광안동', '수영구 민락동',
          '사상구 삼락동', '사상구 모라동', '사상구 덕포동', '사상구 쾌법동', '사상구 감전동', '사상구 주례동', '사상구 학장동', '사상구 엄궁동',
          '기장군 기장읍', '기장군 장안읍', '기장군 정관읍', '기장군 일광면', '기장군 철마면']
categorys = ['카페']

In [11]:
# gu_list = ['중구 중앙동']
# categorys = ['패밀리레스토랑', '패스트푸드']

columns = ['name', 'category', 'address', 'tel', 'time', 'menu', 'image']
df = pd.DataFrame(columns=columns)
review_columns = ['name', 'user_id', 'score', 'review']
df_review = pd.DataFrame(columns=review_columns)

for index, gu_name in enumerate(gu_list):
    for category in categorys:
        print("------------" + gu_name + category + "--------------------")
        time.sleep(random.randint(1, 10))
        check = False
        options = webdriver.ChromeOptions()
        options.add_argument('lang=ko_KR')
        options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36")
        options.add_argument('headless')
        driver = webdriver.Chrome('./chromedriver')  # chromedriver 열기
        driver.get('https://map.kakao.com/')
        search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창
        search_area.send_keys('부산 '+ gu_name + ' ' + category)
        driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
        driver.implicitly_wait(1) # 기다려 주자
        more_page = driver.find_element_by_id("info.search.place.more")
        
        try:
            more_page.send_keys(Keys.ENTER)
            check = True
#             print("눌렀다")
        except:
#             print("못눌렀다")
            pass
#         more_page.send_keys(Keys.ENTER) # 더보기 누르고
#         print(more_page.get_attribute("class").split(' '))
        
#         # 장소더보기 비활성화일때
#         print("보자",more_page.get_attribute("class"))
        
        # 더보기를 누를 수 있으면
        if check == True:
#         if len(more_page.get_attribute("class").split(' ')) > 1:
#             more_page.send_keys(Keys.ENTER) # 더보기 누르고
            time.sleep(random.randint(1, 3))
        
            # 다음 버튼 찾기
            next_btn = driver.find_element_by_id("info.search.page.next")
#             print("이건?", next_btn.get_attribute('class').split(' '))
            has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
            Page = 1

            # 5페이지가 넘을 때
            if has_next:
                while has_next: # 다음 페이지가 있으면 loop
                    driver.implicitly_wait(1) # 기다려 주자
                    page_links = driver.find_elements_by_css_selector("#info\.search\.page a")
                    pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        #             print(pages)
                    for i in range(1, 6):
                        xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                        try:
                            page = driver.find_element_by_xpath(xPath)
                            page.send_keys(Keys.ENTER)
                        except ElementNotInteractableException:
#                             print('End of Page')
                            break;
                        time.sleep(random.randint(1, 10))
                        place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li')
                        for p in place_lists: # WebElement
                            store_html = p.get_attribute('innerHTML')
                            store_info = BeautifulSoup(store_html, "html.parser")
                            place_name = store_info.select('.head_item > .tit_name > .link_name')
                            if len(place_name) == 0:
                                continue # 광고

                            place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                            place_address = store_info.select('.info_item > .addr > p')[0].text
                            place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                            place_tel = store_info.select('.info_item > .contact > span')[0].text
                            place_category = store_info.select('.head_item > span')[0].text

                            time.sleep(random.randint(1, 10))
                            detail = p.find_element_by_css_selector('div.info_item > div.contact > a.moreview')
                            detail.send_keys(Keys.ENTER)

                            driver.switch_to.window(driver.window_handles[-1])
                            driver.implicitly_wait(1) # 기다려 주자

                            menu_lst = []

                            last_height = driver.execute_script("return document.body.scrollHeight")
                            while True:
                                # 스크롤 끝까지 내리기
                                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                                sleep(2)
                                # 스크롤 다운 후 스크롤 높이 다시 가져옴
                                new_height = driver.execute_script("return document.body.scrollHeight")
                                if new_height == last_height:
                                    break
                                last_height = new_height

                            html = driver.page_source
                            soup = BeautifulSoup(html, 'html.parser')
                            menu_all = soup.select('.cont_menu > .list_menu > .nophoto_type')
                            menu_only_all = soup.select('.cont_menu > .list_menu > .menuonly_type')
                            phototype = soup.select('.cont_menu > .list_menu > .photo_type')
        #                     print(menu_all)
                            if len(menu_all) != 0:
                                for menu in menu_all:
                                    menuname = menu.select('.info_menu > .loss_word')[0].text
                                    menuprices = menu.select('.info_menu > .price_menu')
                                    menuprice = ''

                                    if len(menuprices) != 0:
                                        menuprice = menuprices[0].text.split(' ')[1]

                                    menu_lst.append((menuname, menuprice))
                            elif len(menu_only_all) != 0:
                                for menu in menu_only_all:
                                    menuname = menu.select('.info_menu > .loss_word')[0].text
                                    menuprices = menu.select('.info_menu > .price_menu')
                                    menuprice = ''

                                    if len(menuprices) != 0:
                                        menuprice = menuprices[0].text.split(' ')[1]

                                    menu_lst.append((menuname, menuprice))
                            else:
                                for menu in phototype:
                                    menuname = menu.select('.info_menu > .loss_word')[0].text
                                    menuprices = menu.select('.info_menu > .price_menu')
                                    menuprice = ''

                                    if len(menuprices) != 0:
                                        menuprice = menuprices[0].text.split(' ')[1]

                                    menu_lst.append((menuname, menuprice))

                            if place_hour == "":
                                try:
                                    place_hour = soup.select('.location_present > ul > li > span')[0].text
                                except:
                                    place_hour = ""

                            place_photo = ""
                            try:
                                photo = driver.find_element_by_css_selector('span.bg_present')
                                photo_url = photo.get_attribute('style')
                                m = re.search('"(.+?)"', photo_url)
                                if m:
                                    place_photo = m.group(1)
                                else:
                                    place_photo = ""
                            except:
                                place_photo = ""

                            try:
                                page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
                            except:
                                page_num = 0

                            stars = []
                            reviews = []
                            names = []
                            nicknames = []
                            review_lists = soup.select('.list_evaluation > li')

                            if page_num == 0:
                                if len(review_lists) != 0:
                                    t1 = driver.page_source
                                    t2 = BeautifulSoup(t1, "html.parser")
                                    t3 = t2.find(name="div", attrs={"class":"evaluation_review"})
                            #         print(t3)

                                    star = t3.find_all('em',{'class':'num_rate'})
                                    review = t3.find_all('p',{'class':'txt_comment'})
                                    name = t3.find_all('a', {'class': 'link_user'})

                                    stars.extend(star)
                                    reviews.extend(review)
                                    nicknames.extend(name)
                                else:
                                    pass
                            else:
                                for i in range(1, page_num+1):
                                    sleep(1)
                                    t1 = driver.page_source
                                    t2 = BeautifulSoup(t1, "html.parser")
                                    t3 = t2.find(name="div", attrs={"class":"evaluation_review"})
                            #         print(t3)

                                    star = t3.find_all('em',{'class':'num_rate'})
                                    review = t3.find_all('p',{'class':'txt_comment'})
                                    name = t3.find_all('a', {'class': 'link_user'})

                            #         names.append(place)
                                    stars.extend(star)
                                    reviews.extend(review)
                                    nicknames.extend(name)

                                    i += 1

                                    if i > page_num:
                                        break

                                    next_page = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                                    next_page.send_keys(Keys.ENTER)
                                    sleep(1)

                            driver.close()
                            driver.switch_to.window(driver.window_handles[0])

                            df = df.append({
                                'name': place_name,
                                'category': place_category,
                                'address' : place_address,
                                'tel': place_tel,
                                'time': place_hour,
                                'menu': menu_lst,
                                'image': place_photo
                            }, ignore_index=True)

                            names += [place_name] * len(stars)
                            for name, user_id, star, review in zip(names, nicknames, stars, reviews):
                                row = [name, user_id['data-userid'], star.text[0], review.find(name="span").text]
                                series = pd.Series(row, index=df_review.columns)
                                df_review = df_review.append(series, ignore_index=True)

                            print(place_name, len(df))


                    next_btn = driver.find_element_by_id("info.search.page.next")
                    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
                    if not has_next:
                        print('Arrow is Disabled')
                    else:
                        Page += 1
                        next_btn.send_keys(Keys.ENTER)
            # 5페이지가 안넘을 때
            else:
                for i in range(1, 6):
                    xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                    try:
                        page = driver.find_element_by_xpath(xPath)
                        page.send_keys(Keys.ENTER)
                    except ElementNotInteractableException:
                        print('End of Page')
                        break;
                    time.sleep(random.randint(1, 10))
                    place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li')
                    for p in place_lists: # WebElement
                        store_html = p.get_attribute('innerHTML')
                        store_info = BeautifulSoup(store_html, "html.parser")
                        place_name = store_info.select('.head_item > .tit_name > .link_name')
                        if len(place_name) == 0:
                            continue # 광고

                        place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                        place_address = store_info.select('.info_item > .addr > p')[0].text
                        place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                        place_tel = store_info.select('.info_item > .contact > span')[0].text
                        place_category = store_info.select('.head_item > span')[0].text

                        time.sleep(random.randint(1, 10))
                        detail = p.find_element_by_css_selector('div.info_item > div.contact > a.moreview')
                        detail.send_keys(Keys.ENTER)

                        driver.switch_to.window(driver.window_handles[-1])
                        driver.implicitly_wait(1) # 기다려 주자

                        menu_lst = []

                        last_height = driver.execute_script("return document.body.scrollHeight")
                        while True:
                            # 스크롤 끝까지 내리기
                            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                            sleep(2)
                            # 스크롤 다운 후 스크롤 높이 다시 가져옴
                            new_height = driver.execute_script("return document.body.scrollHeight")
                            if new_height == last_height:
                                break
                            last_height = new_height

                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        menu_all = soup.select('.cont_menu > .list_menu > .nophoto_type')
                        menu_only_all = soup.select('.cont_menu > .list_menu > .menuonly_type')
                        phototype = soup.select('.cont_menu > .list_menu > .photo_type')
    #                     print(menu_all)
                        if len(menu_all) != 0:
                            for menu in menu_all:
                                menuname = menu.select('.info_menu > .loss_word')[0].text
                                menuprices = menu.select('.info_menu > .price_menu')
                                menuprice = ''

                                if len(menuprices) != 0:
                                    menuprice = menuprices[0].text.split(' ')[1]

                                menu_lst.append((menuname, menuprice))
                        elif len(menu_only_all) != 0:
                            for menu in menu_only_all:
                                menuname = menu.select('.info_menu > .loss_word')[0].text
                                menuprices = menu.select('.info_menu > .price_menu')
                                menuprice = ''

                                if len(menuprices) != 0:
                                    menuprice = menuprices[0].text.split(' ')[1]

                                menu_lst.append((menuname, menuprice))
                        else:
                            for menu in phototype:
                                menuname = menu.select('.info_menu > .loss_word')[0].text
                                menuprices = menu.select('.info_menu > .price_menu')
                                menuprice = ''

                                if len(menuprices) != 0:
                                    menuprice = menuprices[0].text.split(' ')[1]

                                menu_lst.append((menuname, menuprice))

                        if place_hour == "":
                            try:
                                place_hour = soup.select('.location_present > ul > li > span')[0].text
                            except:
                                place_hour = ""
                        
                        sleep(1)

                        place_photo = ""
                        try:
                            photo = driver.find_element_by_css_selector('span.bg_present')
                            photo_url = photo.get_attribute('style')
                            m = re.search('"(.+?)"', photo_url)
                            if m:
                                place_photo = m.group(1)
                            else:
                                place_photo = ""
                        except:
                            place_photo = ""

                        try:
                            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
                        except:
                            page_num = 0

                        stars = []
                        reviews = []
                        names = []
                        nicknames = []
                        review_lists = soup.select('.list_evaluation > li')

                        if page_num == 0:
                            if len(review_lists) != 0:
                                t1 = driver.page_source
                                t2 = BeautifulSoup(t1, "html.parser")
                                t3 = t2.find(name="div", attrs={"class":"evaluation_review"})
                        #         print(t3)

                                star = t3.find_all('em',{'class':'num_rate'})
                                review = t3.find_all('p',{'class':'txt_comment'})
                                name = t3.find_all('a', {'class': 'link_user'})

                                stars.extend(star)
                                reviews.extend(review)
                                nicknames.extend(name)
                            else:
                                pass
                        else:
                            for i in range(1, page_num+1):
                                t1 = driver.page_source
                                t2 = BeautifulSoup(t1, "html.parser")
                                t3 = t2.find(name="div", attrs={"class":"evaluation_review"})
                        #         print(t3)

                                star = t3.find_all('em',{'class':'num_rate'})
                                review = t3.find_all('p',{'class':'txt_comment'})
                                name = t3.find_all('a', {'class': 'link_user'})

                        #         names.append(place)
                                stars.extend(star)
                                reviews.extend(review)
                                nicknames.extend(name)

                                i += 1

                                if i > page_num:
                                    break

                                next_page = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                                next_page.send_keys(Keys.ENTER)
                                sleep(1)

                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])

                        df = df.append({
                            'name': place_name,
                            'category': place_category,
                            'address' : place_address,
                            'tel': place_tel,
                            'time': place_hour,
                            'menu': menu_lst,
                            'image': place_photo
                        }, ignore_index=True)

                        names += [place_name] * len(stars)
                        for name, user_id, star, review in zip(names, nicknames, stars, reviews):
                            row = [name, user_id['data-userid'], star.text[0], review.find(name="span").text]
                            series = pd.Series(row, index=df_review.columns)
                            df_review = df_review.append(series, ignore_index=True)

                        print(place_name, len(df))
                
        # 못누르는 경우
        elif len(more_page.get_attribute("class").split(' ')) > 1:
            place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                if len(place_name) == 0:
                    continue # 광고

                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_tel = store_info.select('.info_item > .contact > span')[0].text
                place_category = store_info.select('.head_item > span')[0].text

                time.sleep(random.randint(1, 10))
                detail = p.find_element_by_css_selector('div.info_item > div.contact > a.moreview')
                detail.send_keys(Keys.ENTER)

                driver.switch_to.window(driver.window_handles[-1])
                driver.implicitly_wait(1) # 기다려 주자

                menu_lst = []

                last_height = driver.execute_script("return document.body.scrollHeight")
                while True:
                    # 스크롤 끝까지 내리기
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    sleep(2)
                    # 스크롤 다운 후 스크롤 높이 다시 가져옴
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                menu_all = soup.select('.cont_menu > .list_menu > .nophoto_type')
                menu_only_all = soup.select('.cont_menu > .list_menu > .menuonly_type')
                phototype = soup.select('.cont_menu > .list_menu > .photo_type')
#                     print(menu_all)
                if len(menu_all) != 0:
                    for menu in menu_all:
                        menuname = menu.select('.info_menu > .loss_word')[0].text
                        menuprices = menu.select('.info_menu > .price_menu')
                        menuprice = ''

                        if len(menuprices) != 0:
                            menuprice = menuprices[0].text.split(' ')[1]

                        menu_lst.append((menuname, menuprice))
                elif len(menu_only_all) != 0:
                    for menu in menu_only_all:
                        menuname = menu.select('.info_menu > .loss_word')[0].text
                        menuprices = menu.select('.info_menu > .price_menu')
                        menuprice = ''

                        if len(menuprices) != 0:
                            menuprice = menuprices[0].text.split(' ')[1]

                        menu_lst.append((menuname, menuprice))
                else:
                    for menu in phototype:
                        menuname = menu.select('.info_menu > .loss_word')[0].text
                        menuprices = menu.select('.info_menu > .price_menu')
                        menuprice = ''

                        if len(menuprices) != 0:
                            menuprice = menuprices[0].text.split(' ')[1]

                        menu_lst.append((menuname, menuprice))
                        
                if place_hour == "":
                    try:
                        place_hour = soup.select('.location_present > ul > li > span')[0].text
                    except:
                        place_hour = ""

                place_photo = ""
                try:
                    photo = driver.find_element_by_css_selector('span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""

                try:
                    page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
                except:
                    page_num = 0

                stars = []
                reviews = []
                names = []
                nicknames = []
                review_lists = soup.select('.list_evaluation > li')

                if page_num == 0:
                    if len(review_lists) != 0:
                        t1 = driver.page_source
                        t2 = BeautifulSoup(t1, "html.parser")
                        t3 = t2.find(name="div", attrs={"class":"evaluation_review"})
                #         print(t3)

                        star = t3.find_all('em',{'class':'num_rate'})
                        review = t3.find_all('p',{'class':'txt_comment'})
                        name = t3.find_all('a', {'class': 'link_user'})

                        stars.extend(star)
                        reviews.extend(review)
                        nicknames.extend(name)
                    else:
                        pass
                else:
                    for i in range(1, page_num+1):
                        t1 = driver.page_source
                        t2 = BeautifulSoup(t1, "html.parser")
                        t3 = t2.find(name="div", attrs={"class":"evaluation_review"})
                #         print(t3)

                        star = t3.find_all('em',{'class':'num_rate'})
                        review = t3.find_all('p',{'class':'txt_comment'})
                        name = t3.find_all('a', {'class': 'link_user'})

                #         names.append(place)
                        stars.extend(star)
                        reviews.extend(review)
                        nicknames.extend(name)

                        i += 1

                        if i > page_num:
                            break

                        next_page = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                        next_page.send_keys(Keys.ENTER)
                        sleep(1)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

                df = df.append({
                    'name': place_name,
                    'category': place_category,
                    'address' : place_address,
                    'tel': place_tel,
                    'time': place_hour,
                    'menu': menu_lst,
                    'image': place_photo
                }, ignore_index=True)

                names += [place_name] * len(stars)
                for name, user_id, star, review in zip(names, nicknames, stars, reviews):
                    row = [name, user_id['data-userid'], star.text[0], review.find(name="span").text]
                    series = pd.Series(row, index=df_review.columns)
                    df_review = df_review.append(series, ignore_index=True)

                print(place_name, len(df))
                
        driver.close()

------------중구 남포동카페--------------------
채도 1
투썸플레이스 부산남포점 2
탑플로어 3
클라우드밤 4
스타벅스 자갈치역점 5
스타벅스 비프광장점 6
뚜벅스 남포점 7
꿈꾸는다락방 8
펫카페라라랜드 9
미스터힐링 남포동점 10
네루다커피 11
카페슬로 12
코드네임블랙 부산점 13
커핑브레드 남포본점 14
할리스 부산BIFF광장점 15
아수라발발타 남포점 16
이디야커피 부산광복중앙점 17
빽다방 부산광복로점 18
탐앤탐스 부산남포점 19
서정다방 20
하삼동커피 남포점 21
고망고 남포점 22
영커피 남포점 23
양이양이 24
카페부스 25
카페반디나무 26
아마스빈 남포점 27
공차 부산남포점 28
심신프리 부산점 29
오소아 비프점 30
더타임나우 31
매머드익스프레스 부산남포점 32
컴포즈커피 BIFF거리점 33
커피에반하다 부산광복로데오점 34
팔공티 부산남포점 35
더리터 비프광장점 36
다해정 37
차앤 38
모히또바인오션 롯데시네마 대영점 39
카페밈 40
경복커피숖 41
카페051 남포역점 42
꿈꾸는다락방 2호점 43
신한커피숍 44
부일커피숍 45
커피민 46
바니샌드 47
호수커피숍 48
자유공간 49
남도커피숖 50
홀릭에스프레소 51
카페라비아 52
당근나라 53
탑스빈 KEB하나은행점 54
레인보우라이브 55
예지원 56
고구마명가 57
러브보트 58
쉼표셀프힐링카페 59
티맑은 남포점 60
잇커피 자갈치점 61
커피루틴 비프광장점 62
북카페 63
얼리지언스 64
카페지와이 65
아그리나 드레스체험카페 66
마이쥬스 남포점 67
도피오 68
잣나무 69
통카페 70
라무 71
타임커피 72
세시봉 73
블랙레이디 74
까까로운 75
마피아쥬스 남포동점 76
커피반해 77
더테이블 78
굿마우스필 79
부산컨세션 80
카페가비 81
남포당 82
아망떼룸카페 83
쿠오리노 84
스타벅스 부산광복로R점 85
아인스크레페 86
투썸플레이스 부산광복점 87
공차 부산광복점 88
와플칸 89
스타벅스 남포동역점

UnexpectedAlertPresentException: Alert Text: 접속이 원활하지 않습니다.
잠시 후 다시 이용해주세요.
Message: unexpected alert open: {Alert text : 접속이 원활하지 않습니다.
잠시 후 다시 이용해주세요.}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x005E69A3+2582947]
	Ordinal0 [0x0057A6D1+2139857]
	Ordinal0 [0x00473A98+1063576]
	Ordinal0 [0x004C818B+1409419]
	Ordinal0 [0x004B8366+1344358]
	Ordinal0 [0x00495176+1200502]
	Ordinal0 [0x00496066+1204326]
	GetHandleVerifier [0x0078BE02+1675858]
	GetHandleVerifier [0x0084036C+2414524]
	GetHandleVerifier [0x0067BB01+560977]
	GetHandleVerifier [0x0067A8D3+556323]
	Ordinal0 [0x0058020E+2163214]
	Ordinal0 [0x00585078+2183288]
	Ordinal0 [0x005851C0+2183616]
	Ordinal0 [0x0058EE1C+2223644]
	BaseThreadInitThunk [0x75BDFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77D37A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77D37A4E+238]


In [12]:
df.to_csv(path_or_buf='kakao_map_ver5.csv', encoding='utf-8-sig')
df_review.to_csv(path_or_buf='kakao_review_ver5.csv', encoding='utf-8-sig')